# Лабораторная работа №1-2. Парсинг Хабр Карьера

**Вариант: 13**

**Студент:** Мареев Георгий Александрович

**Цель работы:** научиться извлекать данные с веб-страниц, которые подгружают контент динамически с помощью JavaScript. Освоить использование библиотеки `Selenium` для управления браузером и языка `XPath` для точного поиска элементов на странице.

**Бизнес-кейс:** анализ рынка труда для разработчикво: вакансии "Python Developer".

**Объект парсинга:**  [Хабр Карьера](https://career.habr.com), с фильтрами "Python" и "Django".

**Аналитическая задача:** Собрать компании и зарплатные вилки. Сравнить медианную зарплату с общим рынком Python-разработчиков


Изучение страницы, её элементов для парсинга
search:
<div class="search-input">

<input class = "text-input_input">
filters:
<span class> "filter-item_title"

#### Работа с SQLite

После сбора и первичной очистки данных в Pandas DataFrame, их необходимо сохранить в локальную базу данных SQLite. Это стандартная практика для хранения структурированных данных, позволяющая выполнять сложные запросы без повторного парсинга сайта.